In [ ]:
import os
import glob
import pandas as pd

def aggregate_experiment_results(dataset_name: str):
    folder_path = f"C:/Users/romai/Desktop/gnn/gnn_pu/NNIF-GNN/{dataset_name}_experimentations"
    csv_files = glob.glob(os.path.join(folder_path, '*.csv'))
    
    aggregated_results = []
    
    for file in csv_files:
        print(f"Processing {file}...")
        try:
            df = pd.read_csv(file)
            
            # Check if the dataframe has exactly 5 rows
            if len(df) == 5:
                aggregated = {}
                for col in df.columns:
                    if pd.api.types.is_numeric_dtype(df[col]):
                        aggregated[col] = df[col].mean()
                    else:
                        mode_val = df[col].mode()
                        aggregated[col] = mode_val.iloc[0] if not mode_val.empty else None
                aggregated_results.append(aggregated)
        except Exception as e:
            print(f"Error processing {file}: {e}")
    
    aggregated_df = pd.DataFrame(aggregated_results)
    print(aggregated_df)
    aggregated_df=aggregated_df.sort_values(by="f1", ascending=False)
    aggregated_df.to_csv(f"{dataset_name}_aggregated_results.csv", index=False)
    return aggregated_df


In [6]:
df=aggregate_experiment_results("citeseer")

Processing C:/Users/romai/Desktop/gnn/gnn_pu/NNIF-GNN/citeseer_experimentations\citeseer_scar_sampling_nnif_batch_cluster_1103092822.csv...
Processing C:/Users/romai/Desktop/gnn/gnn_pu/NNIF-GNN/citeseer_experimentations\citeseer_scar_sampling_nnif_batch_cluster_1103095136.csv...
Processing C:/Users/romai/Desktop/gnn/gnn_pu/NNIF-GNN/citeseer_experimentations\citeseer_scar_sampling_nnif_batch_cluster_1103095830.csv...
Processing C:/Users/romai/Desktop/gnn/gnn_pu/NNIF-GNN/citeseer_experimentations\citeseer_scar_sampling_nnif_batch_cluster_1103100718.csv...
Processing C:/Users/romai/Desktop/gnn/gnn_pu/NNIF-GNN/citeseer_experimentations\citeseer_scar_sampling_nnif_batch_cluster_1103100747.csv...
Processing C:/Users/romai/Desktop/gnn/gnn_pu/NNIF-GNN/citeseer_experimentations\citeseer_scar_sampling_nnif_batch_cluster_1103101517.csv...
Processing C:/Users/romai/Desktop/gnn/gnn_pu/NNIF-GNN/citeseer_experimentations\citeseer_scar_sampling_nnif_batch_cluster_1103101938.csv...
Processing C:/Users/

In [9]:
df.sort_values(by="f1", ascending=False).head(30)

,K,layers,hidden_channels,out_channels,norm,lr,treatment,dropout,ratio,seed,...,batch_size,rate_pairs,reliable_mini_batch,clusters,pos_weight,accuracy,f1,recall,precision,losses
301,5.0,1.0,128.0,256.0,NaN,0.002146,removal,0.0,0.145865,366.6,...,10.0,1.0,1.0,100.0,NaN,0.945056,0.866872,0.849073,0.885475,"[8.539441049098969, 7.715812385082245, 7.89558..."
129,5.0,1.0,256.0,64.0,NaN,0.007393,removal,0.0,0.126253,366.6,...,15.0,9.0,1.0,200.0,0.233462,0.945236,0.865679,0.837946,0.895430,"[3.7778583616018295, 3.1326910853385925, 3.113..."
300,5.0,1.0,128.0,256.0,NaN,0.001916,removal,0.0,0.143588,366.6,...,10.0,1.0,1.0,100.0,NaN,0.944274,0.865292,0.849358,0.881966,"[8.54531705379486, 7.682579517364502, 7.882440..."
41,5.0,1.0,256.0,64.0,NaN,0.006777,removal,0.0,0.103433,366.6,...,15.0,10.0,1.0,200.0,0.232942,0.946078,0.864230,0.814551,0.920365,"[3.884629398584366, 3.2001337856054306, 3.1727..."
297,5.0,1.0,128.0,256.0,NaN,0.002186,removal,0.0,0.146151,366.6,...,10.0,1.0,1.0,100.0,NaN,0.944034,0.864098,0.844508,0.884701,"[8.529613554477692, 7.709229946136475, 7.89154..."
23,5.0,1.0,256.0,64.0,NaN,0.008383,removal,0.0,0.126920,366.6,...,15.0,10.0,1.0,200.0,0.286657,0.944755,0.864068,0.833381,0.897105,"[4.328235983848572, 3.6192904710769653, 3.5737..."
273,3.0,1.0,128.0,256.0,NaN,0.003239,removal,0.0,0.139025,366.6,...,10.0,3.0,1.0,100.0,NaN,0.944695,0.863788,0.832240,0.897847,"[6.76313978433609, 6.1002733409404755, 6.18579..."
84,5.0,1.0,256.0,64.0,NaN,0.005743,removal,0.0,0.107158,366.6,...,15.0,9.0,1.0,200.0,0.211875,0.945597,0.863755,0.818545,0.914279,"[3.6805271357297897, 3.0237891525030136, 3.002..."
116,5.0,1.0,256.0,64.0,NaN,0.008699,removal,0.0,0.134557,366.6,...,15.0,9.0,1.0,200.0,0.255802,0.944394,0.863596,0.835663,0.893486,"[3.903626501560211, 3.254894971847534, 3.22112..."
134,5.0,1.0,256.0,64.0,NaN,0.007104,removal,0.0,0.125631,366.6,...,15.0,8.0,1.0,200.0,0.265115,0.944094,0.863576,0.840228,0.888466,"[4.063809186220169, 3.4205896109342575, 3.3808..."


In [16]:
import numpy as np
import pandas as pd
path=r"C:\Users\romai\Desktop\elliptic_bitcoin_dataset"
# Define path to CSV file
csv_path = path + r"\elliptic_txs_classes.csv"

# Load data with pandas for better handling
df = pd.read_csv(csv_path)

# Check for missing or malformed values explicitly
missing_rows = df[df.isnull().any(axis=1)]

if not missing_rows.empty:
    print("Found missing or malformed rows at indices:")
    print(missing_rows.index.tolist())
else:
    print("No missing rows detected with pandas.")

# Load data using numpy for original processing
y_str = np.loadtxt(csv_path,
                   delimiter=",", skiprows=1, usecols=(0,1), dtype=str)

print("Loaded numpy data shape:", y_str.shape)
print("Pandas DataFrame shape:", df.shape)

# Identify missing rows by comparing pandas and numpy lengths
expected_rows = df.shape[0]
loaded_rows = y_str.shape[0]

if loaded_rows < expected_rows:
    print(f"Missing {expected_rows - y_str.shape[0]} rows in numpy load.")
    missing_indices = set(df.index) - set(range(y_str.shape[0]))
    print("Missing row indices:", missing_rows)

# Verify exact missing rows if needed
if len(df) != len(y_str):
    missing_rows = df[~df.index.isin(range(len(y_str)))]
    print("Detailed missing rows:")
    print(missing_rows)


No missing rows detected with pandas.
Loaded numpy data shape: (203765, 2)
Pandas DataFrame shape: (203769, 2)
Missing 4 rows in numpy load.
Missing row indices: Empty DataFrame
Columns: [txId, class]
Index: []
Detailed missing rows:
             txId    class
203765  158577750  unknown
203766  158375402        1
203767  158654197  unknown
203768  157597225  unknown
